In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, sqlite3, requests
from zipfile import ZipFile
from io import BytesIO
# !pip install ipython-sql
pd.set_option("display.max_columns", None)

# Download data from WB

In [14]:
# download dataset from the world bank
print("Starting download datasets...")
url = "https://databank.worldbank.org/data/download/IDS_CSV.zip"
file_name = url.split("/")[-1]
res = requests.get(url)
zf = ZipFile(BytesIO(res.content))
zf.extractall("../data")
print(f"{file_name} downloaded and unzipped!")

Starting download datasets...
IDS_CSV.zip downloaded and unzipped!


In [24]:
os.listdir("../data")

['IDS_Footnotes.csv',
 '.DS_Store',
 'IDS_Series-TimeMetaData.csv',
 'IDS_ALLCountries_Data.csv',
 'IDS_Country-SeriesMetaData.csv',
 'IDS_CountryMetaData.csv',
 'IDS_SeriesMetaData.csv']

# EDA the dataset

In [27]:
int_debt_df = pd.read_csv("../data/IDS_ALLCountries_Data.csv")
int_debt_df.shape

(77389, 65)

In [28]:
int_debt_df.head()

,Country Name,Country Code,Counterpart-Area Name,Counterpart-Area Code,Series Name,Series Code,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028
0,Afghanistan,AFG,World,WLD,Average grace period on new external debt comm...,DT.GPA.DPPG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.2500,8.0833,9.9575,0.0,7.8333,0.0,0.0,0.0,29.8333,0.0,0.0,19.8356,0.0,17.9041,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,World,WLD,Average grace period on new external debt comm...,DT.GPA.OFFT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.2500,8.0833,9.9575,0.0,7.8333,0.0,0.0,0.0,29.8333,0.0,0.0,19.8356,0.0,17.9041,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,AFG,World,WLD,Average grace period on new external debt comm...,DT.GPA.PRVT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,AFG,World,WLD,Average grant element on new external debt com...,DT.GRE.DPPG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.6176,43.0248,58.0061,0.0,21.2119,0.0,0.0,0.0,81.9289,0.0,0.0,64.2646,0.0,73.0790,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,AFG,World,WLD,Average grant element on new external debt com...,DT.GRE.OFFT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.6176,43.0248,58.0061,0.0,21.2119,0.0,0.0,0.0,81.9289,0.0,0.0,64.2646,0.0,73.0790,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The dataset appears to be in a wide format with each year data value in each column. As I'm used to working with panel data, I want to convert this into a long format in which each year value has a row. The combination of country code, series code, and year would create a unique key.

In [29]:
int_debt_df.columns

Index(['Country Name', 'Country Code', 'Counterpart-Area Name',
       'Counterpart-Area Code', 'Series Name', 'Series Code', '1970', '1971',
       '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980',
       '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989',
       '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998',
       '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007',
       '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016',
       '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024', '2025',
       '2026', '2027', '2028'],
      dtype='object')

In [32]:
int_debt_long_df = pd.melt(int_debt_df, id_vars=['Country Name', 'Country Code', 'Counterpart-Area Name',
       'Counterpart-Area Code', 'Series Name', 'Series Code'], var_name="year", value_name="debt_amount")

In [33]:
int_debt_long_df.head()

,Country Name,Country Code,Counterpart-Area Name,Counterpart-Area Code,Series Name,Series Code,year,debt_amount
0,Afghanistan,AFG,World,WLD,Average grace period on new external debt comm...,DT.GPA.DPPG,1970,NaN
1,Afghanistan,AFG,World,WLD,Average grace period on new external debt comm...,DT.GPA.OFFT,1970,NaN
2,Afghanistan,AFG,World,WLD,Average grace period on new external debt comm...,DT.GPA.PRVT,1970,NaN
3,Afghanistan,AFG,World,WLD,Average grant element on new external debt com...,DT.GRE.DPPG,1970,NaN
4,Afghanistan,AFG,World,WLD,Average grant element on new external debt com...,DT.GRE.OFFT,1970,NaN


In [34]:
del int_debt_df
int_debt_long_df = int_debt_long_df.rename(columns={'Country Name':"country_name", 'Country Code':"country_code", 'Counterpart-Area Name':"counterpart_area_name",
       'Counterpart-Area Code':"counterpart_area_code", 'Series Name':"series_name", 'Series Code':"series_code"})

In [35]:
int_debt_long_df.columns

Index(['country_name', 'country_code', 'counterpart_area_name',
       'counterpart_area_code', 'series_name', 'series_code', 'year',
       'debt_amount'],
      dtype='object')

# Create a new database
Here I create a new sqlite3 db from the IDS_ALLCountries_Data.csv. Since my purpose here is to do some analysis using sql

In [45]:
conn = sqlite3.connect("../data/int_debt.db")
cur = conn.cursor()

In [46]:
cur.execute('''CREATE TABLE debts (country_name text, country_code int, counterpart_area_name text, counterpart_area_code int, series_name text, series_code int, year int, debt_amount float)''')

In [47]:
int_debt_long_df.to_sql("debts", conn, if_exists="append", index=False)

4565951

Now I have a db in my data folder. Let's do some analysis using sql.

In [56]:
%load_ext sql

In [60]:
%sql sqlite:///../data/int_debt.db

Get column names from the table. This is quite specific to sqlite

In [83]:
%%sql
PRAGMA table_info("debts")

 * sqlite:///../data/int_debt.db
Done.


cid,name,type,notnull,dflt_value,pk
0,country_name,TEXT,0,None,0
1,country_code,INT,0,None,0
2,counterpart_area_name,TEXT,0,None,0
3,counterpart_area_code,INT,0,None,0
4,series_name,TEXT,0,None,0
5,series_code,INT,0,None,0
6,year,INT,0,None,0
7,debt_amount,float,0,None,0


Now get all the country names and codes

In [84]:
%%sql 
SELECT DISTINCT country_name, country_code
FROM debts

 * sqlite:///../data/int_debt.db
Done.


country_name,country_code
Afghanistan,AFG
Albania,ALB
Algeria,DZA
Angola,AGO
Argentina,ARG
Armenia,ARM
Azerbaijan,AZE
Bangladesh,BGD
Belarus,BLR
Belize,BLZ
